In [ ]:
import weave
from weave.ecosystem.all import langchain
weave.use_frontend_devmode()

# Goals:
#   - solve math problems
#   - log behavior of 2 OpenAI models and Claude
#   - try 2 different prompts each
#   - try 2 different temperatures each

In [ ]:
problems = weave.save([
    {'problem': 'integral of 2^x'},
    {'problem': 'sqrt of pi to 10 decimals'}],
    #{'problem': 'how many city blocks are there in boston?'}],
    'math_problems')

In [ ]:
problems

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.chat_models import ChatAnthropic
from langchain.prompts import PromptTemplate
from langchain.llms import OpenAI
from langchain.chains import LLMChain

#llm = OpenAI(temperature=0.9)
blank_prompt = PromptTemplate(
    input_variables=["problem"],
    template="{problem}",
)
prompt = PromptTemplate(
    input_variables=["problem"],
    template="solve this problem step by step: {problem}?",
)

#LLMChain(llm=llm, prompt=prompt)

chat_openai = ChatOpenAI()
chat_anthropic = ChatAnthropic(temperature=0.7)
models = [
    LLMChain(llm=chat_openai, prompt=blank_prompt),
    LLMChain(llm=chat_anthropic, prompt=blank_prompt),
    LLMChain(llm=chat_openai, prompt=prompt)]
prompts = ['solve this problem: ', 'solve this problem step by step: ']
ms = weave.save(models, 'models')


In [ ]:
print(ms)

In [ ]:
problems.map(lambda prob: ms.run(prob['problem'])).flatten()

In [ ]:
questions

In [ ]:
results = []
for model in models:
    for prompt in prompts:
        for problem in weave.use(problems):
            results.append({
                'model': model,
                'temp': temp,
                'prompt': prompt,
                'problem': problem,
                'result': weave.use(model.predict(prompt + problem['problem']))})
weave.save(results)

In [ ]:
weave.get_results(chat_openai.predict)

In [ ]:
weave.save(results)

In [ ]:
chat_openai = weave.save(ChatOpenAI(temperature=0.7), 'chat_model')

In [ ]:
chat_anthropic = weave.save(ChatAnthropic(temperature=0.7), 'achat_model')

In [ ]:

def eval_model(model, questions):
    answers = []
    for q in weave.use(questions):
        a = weave.use(model.predict(q['question']))
        answers.append({'model': model, 'question': q['question'], 'answer': a})
    return weave.save(answers, '%s-answers' % model.type.name)

In [ ]:
eval_model(chat_anthropic, questions)

In [ ]:
weave.ops.get('local-artifact:///run-BaseChatModel-predict:latest/obj')